In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Uterine_Corpus_Endometrial_Carcinoma"
cohort = "GSE32507"

# Input paths
in_trait_dir = "../../input/GEO/Uterine_Corpus_Endometrial_Carcinoma"
in_cohort_dir = "../../input/GEO/Uterine_Corpus_Endometrial_Carcinoma/GSE32507"

# Output paths
out_data_file = "../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/GSE32507.csv"
out_gene_data_file = "../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/gene_data/GSE32507.csv"
out_clinical_data_file = "../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/clinical_data/GSE32507.csv"
json_path = "../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE32507_family.soft.gz', 'GSE32507_series_matrix.txt.gz']
Identified SOFT files: ['GSE32507_family.soft.gz']
Identified matrix files: ['GSE32507_series_matrix.txt.gz']

Background Information:
!Series_title	"Expression profile of carcinosarcoma (CS), endometrioid adenocarcinoma (EC) and sarcoma (US) of uterine corpus"
!Series_summary	"To examine the simlarity of CS, EC and US, we performed microarray analysis of frozen tissues of 46 patients (14 CS, 24 EC and 8 US)."
!Series_overall_design	"Frozen tissues of 46 patients (14CS, 24EC and 8US) were subjected to cDNA microarray analysis."

Sample Characteristics Dictionary:
{0: ['tissue: carcinosarcoma', 'tissue: endometrioid adenocarcinoma', 'tissue: sarcoma'], 1: ['carcinosarcoma status: : heterologous', 'carcinosarcoma status: : homologous', nan]}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check availability of gene expression data
# This dataset seems to contain gene expression data based on the background information
# mentioning "cDNA microarray analysis", so set is_gene_available to True
is_gene_available = True

# Variable availability and data type conversion
# 1. Trait availability: based on the sample characteristics, tissue type is at key 0
# which differentiates between carcinosarcoma, endometrioid adenocarcinoma, and sarcoma
trait_row = 0

# 2. Age data: not available in sample characteristics
age_row = None

# 3. Gender data: not available, and since this is a study about uterine corpus,
# we can assume all patients are female (but we'll set it as unavailable since it's a constant)
gender_row = None

# Define conversion functions for each variable
def convert_trait(value):
    """Convert the trait value to a binary variable.
    Since we're focused on Uterine_Corpus_Endometrial_Carcinoma, we'll consider
    'endometrioid adenocarcinoma' as our positive class (1) and other types as negative (0).
    """
    if pd.isna(value):
        return None
    
    # Extract the value if it contains a colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'endometrioid adenocarcinoma' in value.lower():
        return 1
    else:  # carcinosarcoma or sarcoma
        return 0

def convert_age(value):
    """Placeholder function for converting age."""
    return None  # Not used as age data is not available

def convert_gender(value):
    """Placeholder function for converting gender."""
    return None  # Not used as gender data is not available

# Check if trait data is available
is_trait_available = trait_row is not None

# Save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available:
    try:
        # Load all sample characteristics from the matrix file
        matrix_file = f"{in_cohort_dir}/GSE32507_series_matrix.txt.gz"
        
        # Create a dictionary to store sample information
        sample_data = {}
        current_sample_idx = -1
        sample_ids = []
        
        with gzip.open(matrix_file, 'rt') as file:
            for line in file:
                line = line.strip()
                
                # Extract sample GEO IDs
                if line.startswith('!Sample_geo_accession'):
                    sample_ids = line.split('\t')[1:]
                    for idx, sample_id in enumerate(sample_ids):
                        sample_data[sample_id] = {}
                
                # Extract characteristics
                elif line.startswith('!Sample_characteristics_ch1'):
                    characteristics = line.split('\t')[1:]
                    
                    # Match each characteristic to its corresponding sample
                    for idx, characteristic in enumerate(characteristics):
                        if idx < len(sample_ids):
                            sample_id = sample_ids[idx]
                            
                            # Append to list of characteristics for this sample
                            if 'characteristics' not in sample_data[sample_id]:
                                sample_data[sample_id]['characteristics'] = []
                            
                            sample_data[sample_id]['characteristics'].append(characteristic)
                
                # If we've processed all sample data, stop reading
                elif line.startswith('!series_matrix_table_begin'):
                    break
        
        # Create a DataFrame to represent our clinical data
        clinical_rows = []
        
        # For each row in the trait_row (key 0 in the sample characteristics)
        for sample_id, data in sample_data.items():
            if 'characteristics' in data and len(data['characteristics']) > trait_row:
                trait_value = data['characteristics'][trait_row]
                clinical_rows.append({
                    'Sample_ID': sample_id,
                    'characteristics_ch1': trait_value
                })
        
        clinical_df = pd.DataFrame(clinical_rows)
        
        if not clinical_df.empty:
            # Print original values for debugging
            print("Original trait values:")
            for val in clinical_df['characteristics_ch1'].unique():
                print(f"  {val}")
            
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_df,
                trait=trait,
                trait_row=0,  # Use 0 here because we've already extracted the trait row
                convert_trait=convert_trait,
                age_row=None,
                convert_age=None,
                gender_row=None,
                convert_gender=None
            )
            
            # Preview the data
            preview = preview_df(selected_clinical_df)
            print("Preview of selected clinical features:")
            print(preview)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the clinical data
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("No clinical data found in the matrix file.")
    except Exception as e:
        print(f"Error extracting or saving clinical features: {e}")
        import traceback
        traceback.print_exc()


Original trait values:
  "tissue: carcinosarcoma"
  "tissue: endometrioid adenocarcinoma"
  "tissue: sarcoma"
Preview of selected clinical features:
{'Sample_ID': [0.0], 'characteristics_ch1': [0.0]}
Clinical data saved to ../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/clinical_data/GSE32507.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '(+)eQC-39', '(+)eQC-41',
       '(+)eQC-42', '(-)3xSLv1', 'A_23_P100001', 'A_23_P100011',
       'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100092'],
      dtype='object', name='ID')

Gene expression data shape: (41073, 46)


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data
# The identifiers (e.g., "A_23_P100001") appear to be Agilent microarray probe IDs, not standard human gene symbols
# These probe IDs need to be mapped to official gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
                 GSM804806  GSM804807  GSM804808  GSM804809  GSM804810
ID                                                                    
(+)E1A_r60_1      0.187544   1.125378   0.308133   1.549022   0.297386
(+)E1A_r60_3     -0.057653   0.098557  -0.019575   2.112438   0.290960
(+)E1A_r60_a104   0.309965   0.280072  -0.410076   1.748169  -0.370941
(+)E1A_r60_a107   0.291783   1.178800  -0.036704   1.191367   0.090694
(+)E1A_r60_a135   0.274253   1.303301   0.063972   1.639965   0.304410



Platform information:
!Series_title = Expression profile of carcinosarcoma (CS), endometrioid adenocarcinoma (EC) and sarcoma (US) of uterine corpus
!Platform_title = Agilent-014850 Whole Human Genome Microarray 4x44K G4112F (Probe Name version)
!Platform_description = This multi-pack (4X44K) formatted microarray represents a compiled view of the human genome as it is understood today. The sequence information used to design this product was derived from a broad survey of well known sources such as RefSeq, Goldenpath, Ensembl, Unigene and others. The resulting view of the human genome covers 41K unique genes and transcripts which have been verified and optimized by alignment to the human genome assembly and by Agilent's Empirical Validation process.
!Platform_description =
!Platform_description = *** The ID column includes the Agilent Probe Names. A different version of this platform with the Agilent Feature Extraction feature numbers in the ID column is assigned accession number GPL4


Gene annotation columns:
['ID', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']

Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptos

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on previous output, the 'ID' column in gene_annotation contains the probe IDs 
# that match the indices in gene_data, and 'GENE_SYMBOL' contains the gene symbols we need

# Check both columns to verify they exist and are appropriate for mapping
print("Sample of IDs from gene_annotation:")
print(gene_annotation['ID'].head())

print("\nSample of IDs from gene_data:")
print(gene_data.index[:5])

print("\nSample of GENE_SYMBOL from gene_annotation:")
print(gene_annotation['GENE_SYMBOL'].head())

# 2. Get gene mapping dataframe with ID and GENE_SYMBOL columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')
print("\nGene mapping dataframe preview:")
print(gene_mapping.head())
print(f"Gene mapping shape: {gene_mapping.shape}")

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("\nGene expression data preview after mapping:")
print(gene_data.head())
print(f"Gene expression data shape after mapping: {gene_data.shape}")

# Normalized gene symbols for consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene expression data after normalizing gene symbols:")
print(gene_data.head())
print(f"Final gene expression data shape: {gene_data.shape}")

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Sample of IDs from gene_annotation:
0    A_23_P100001
1    A_23_P100011
2    A_23_P100022
3    A_23_P100056
4    A_23_P100074
Name: ID, dtype: object

Sample of IDs from gene_data:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135'],
      dtype='object', name='ID')

Sample of GENE_SYMBOL from gene_annotation:
0    FAM174B
1      AP3S2
2       SV2B
3     RBPMS2
4       AVEN
Name: GENE_SYMBOL, dtype: object

Gene mapping dataframe preview:
             ID     Gene
0  A_23_P100001  FAM174B
1  A_23_P100011    AP3S2
2  A_23_P100022     SV2B
3  A_23_P100056   RBPMS2
4  A_23_P100074     AVEN
Gene mapping shape: (30936, 2)

Gene expression data preview after mapping:
          GSM804806  GSM804807  GSM804808  GSM804809  GSM804810  GSM804811  \
Gene                                                                         
A1BG      -0.856747  -2.371038   1.810420   4.458369  -1.614460  -2.098005   
A1BG-AS1   0.112597  -2.545402   0.345880   2.


Gene expression data after normalizing gene symbols:
          GSM804806  GSM804807  GSM804808  GSM804809  GSM804810  GSM804811  \
Gene                                                                         
A1BG      -0.856747  -2.371038   1.810420   4.458369  -1.614460  -2.098005   
A1BG-AS1   0.112597  -2.545402   0.345880   2.294041  -1.484570  -2.047867   
A1CF      -0.829145   2.310278   0.408321   3.008061  -0.764084  -0.462802   
A2M       -1.598132   1.704536  -1.966787   2.845671  -0.677535  -1.352631   
A2ML1     -0.518708  -0.038120   0.456291   0.202503   0.341105   0.115908   

          GSM804812  GSM804813  GSM804814  GSM804815  ...  GSM804842  \
Gene                                                  ...              
A1BG      -1.576633   0.529343  -0.418982  -1.422590  ...  -0.535830   
A1BG-AS1  -1.833521   0.289472  -0.083492  -0.891364  ...   0.349333   
A1CF      -0.823997   0.913546   1.887642   2.264761  ...   0.949150   
A2M        0.799846   2.531783  -1.6247


Gene expression data saved to ../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/gene_data/GSE32507.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (18244, 46)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']


Normalized gene data saved to ../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/gene_data/GSE32507.csv
Loaded clinical data:
           characteristics_ch1
Sample_ID                     
0.0                        0.0
Transposed clinical data to correct format:
Sample_ID            0.0
characteristics_ch1  0.0
Number of common samples between clinical and genetic data: 0
Clinical data index: ['characteristics_ch1']
Gene data columns: ['GSM804806', 'GSM804807', 'GSM804808', 'GSM804809', 'GSM804810', '...']
Extracted 46 GSM IDs from gene data.
Created new clinical data with matching sample IDs:
           Uterine_Corpus_Endometrial_Carcinoma
GSM804806                                     1
GSM804807                                     1
GSM804808                                     1
GSM804809                                     1
GSM804810                                     1
Gene data shape for linking (samples as rows): (46, 18244)
Linked data shape: (46, 18245)
Linked data

Linked data shape after handling missing values: (46, 18245)
For the feature 'Uterine_Corpus_Endometrial_Carcinoma', the least common label is '1' with 14 occurrences. This represents 30.43% of the dataset.
The distribution of the feature 'Uterine_Corpus_Endometrial_Carcinoma' in this dataset is fine.

Is trait biased: False
A new JSON file was created at: ../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/cohort_info.json
Data quality check result: Usable


Linked data saved to ../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/GSE32507.csv
